In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import local_binary_pattern, hog
from skimage.filters import gabor
from skimage.transform import resize
from skimage.util import img_as_ubyte
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

## **Ekstrasi Fitur**

In [2]:
# Ekstraksi fitur HOG
def extract_hog_features(img, size=(200, 200)):
    img_resized = resize(img, size, anti_aliasing=True)
    features, _ = hog(
        img_resized, 
        orientations=9, 
        pixels_per_cell=(8, 8), 
        cells_per_block=(2, 2),
        block_norm='L2-Hys', 
        visualize=True,
        channel_axis=None
    )
    return features

# Ekstraksi fitur LBP
def extract_lbp_features(img, size=(200, 200)):
    img_resized = cv2.resize(img, size)
    img_resized = img_as_ubyte(img_resized)
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(img_resized, n_points, radius, method='uniform')
    
    hist, _ = np.histogram(
        lbp.ravel(),
        bins=np.arange(0, n_points + 3),
        range=(0, n_points + 2)
    )
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

# Ekstraksi fitur Gabor
def extract_gabor_features(img, size=(200, 200)):
    img_resized = resize(img, size, anti_aliasing=True)
    filters = []
    for theta in (0, np.pi / 4, np.pi / 2, 3 * np.pi / 4):  # Sudut orientasi
        for frequency in (0.1, 0.2, 0.3):  # Frekuensi Gabor
            real, _ = gabor(img_resized, frequency=frequency, theta=theta)
            filters.append(np.mean(real))
            filters.append(np.std(real))
    return filters

In [3]:
# Gabungkan semua fitur
def extract_features_from_image(img_path, size=(200, 200)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return np.array([])  # Jika gambar tidak valid, return array kosong
    
    # Ekstraksi fitur
    hog_features = extract_hog_features(img, size)
    lbp_features = extract_lbp_features(img, size)
    gabor_features = extract_gabor_features(img, size)
    
    # Gabungkan semua fitur
    return np.hstack([hog_features, lbp_features, gabor_features])

# Akurasi

In [5]:
import os
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score

# Fungsi untuk membaca dataset lokal dan menyiapkan filepath serta label
def load_dataset(folder_path):
    data = []
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(root, file_name)
                # Ekstrak label dari angka sebelum ekstensi
                label = int(file_name.split('_')[-1].split('.')[0])
                data.append((file_path, label))  # Path dan label
    return pd.DataFrame(data, columns=['filepath', 'label'])

# Fungsi untuk memproses dataset dan menghitung akurasi
def evaluate_model(folder_path, model):
    data = load_dataset(folder_path)
    y_true = []
    y_pred = []

    for _, row in data.iterrows():
        image_path = row['filepath']
        true_label = row['label']
        
        # Ekstraksi fitur gambar
        features = extract_features_from_image(image_path)
        features = features.reshape(1, -1)  # Reshape untuk model

        # Prediksi dengan augmentasi dan tanpa augmentasi
        predicted_label_original = model.predict(features)[0]

        # Menambahkan hasil prediksi ke daftar
        y_true.append(true_label)
        y_pred.append(predicted_label_original)

    # Hitung akurasi
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy, y_true, y_pred

# Path dataset lokal
train_folder = 'dataset/final_dataset/train'
test_folder = 'dataset/final_dataset/test'

# Memuat model
model_path = "model/xgboost_hog_lbp_model_acc_0.66.pkl"
model_xgb = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_xgb)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 17.09%
